In [1]:
import pandas as pd
import numpy as np

In [28]:
dfStrong = pd.read_csv('DA_OilTrends - Oil_Companies.csv', dtype=str)
dfWeak = pd.read_csv('DA_OilTrends - Other_Industries.csv',dtype=str)

companies_df = pd.read_csv('oil_companies.csv')


In [29]:
### List of industries we believe are highly dependent on crude oil

oil_industries = list(set(companies_df["Industry"]))
for industry in oil_industries:
    print(industry)

Refineries/ Petro-Products
Fibres & Plastics
Agrochemicals
Oil Marketing & Distribution
Plastic Products
Oil Equipment & Services
Petrochemicals
Integrated Oil & Gas


In [10]:
### List of companies under each sector available for trend analysis.
industry_company_map = dict()
# list of companies with no "NOT AVAILABLE"
avail_comp = [i for i in list(dfStrong.columns) if "NOT AVAILABLE" not in list(dfStrong[i])]
for oil_industry in oil_industries:
    df = companies_df[companies_df.Industry.str.contains(oil_industry)]
    industry_company_map[oil_industry] = list(set(list(df["Security Id"])) & set(avail_comp))
    
for i in industry_company_map.keys():
    l = industry_company_map[i]
    #Industry, Number of companies in that industry to analyse
    print(i, len(l))
    for j in l:
        print(j)
    print("\n")

Refineries/ Petro-Products 7
GOACARBON
HINDPETRO
SOTL
BPCL
CHENNPETRO
MRPL
PANAMAPET


Fibres & Plastics 7
AMCOIND
JBFIND
POLYLINK
DVL
VINYLINDIA
HINFLUR
INDIANACRY


Agrochemicals 19
KILPEST
JUBLINDS
PHYTO
NACLIND
MONSANTO
PUNJABCHEM
UPL
ASTEC
EXCELCROP
BHAGCHEM
DHANUKA
BAYERCROP
PIIND
INSECTICID
AIMCOPEST
ATUL
SUCROSA
BHARATRAS
RALLIS


Oil Marketing & Distribution 8
CASTROLIND
TIDEWATER
PETRONET
GOCLCORP
IOC
GULFPETRO
AEGISLOG
CONFIPET


Plastic Products 38
ACRYSIL
KINGFA
RDBRL
KISAN
FLEXITUFF
JISLJALEQS
JISLDVREQS
WIMPLAST
PRIMAPLA
ASHISHPO
ARROWGREEN
KRITIIND
RESPONIND
ASTRAL
NATPLAS
DUTRON
FENOPLAS
APOLLOPIPES
TIMETECHNO
HINDADH
BLOOM
TIJARIA
MAHEPC
SIGNETIND
TPLPLAST
TAINWALCHM
ECOPLAST
TEXMOPIPES
SUPREMEIND
NILKAMAL
BIOPAC
RUNGTAIR
SRIKPRIND
MACPLASQ
CAPRIHANS
FINOLEXIND
INTLCONV
KKALPANAIND


Oil Equipment & Services 4
OILCOUNTUB
ASIANOI
DOLPHINOFF
ALPHAGEO


Petrochemicals 5
RAIN
DCW
MANALIPETC
AGARIND
SUPPETRO


Integrated Oil & Gas 2
DEEPIND
RELIANCE




### Correlating the max trend with the crude oil trend

What is max trend ?

Company 1 | Company 2 | Company 3 | Max Trend

     1   |     0   |     1   |    1

Therefore, if crude oil trend is [0,1,-1,0,1,0,0] and max trend is [0,-1,0,0,0,0,-1] and hypothesis is (0,0,0) then, industry score = 5/7 = 0.7142



In [23]:
import itertools
def correlation_trend_analyse_max(industry, period, threshold = 0.7):
    print(industry, len(industry_company_map[industry]))
    df = dfStrong[['Year', 'CRUDEOIL']+industry_company_map[industry]]
    crude_oil_trend = list(df['CRUDEOIL'])[:133]
    dfIndustryTrends = df[industry_company_map[industry]]
    
    l = [0, 1, -1]
    patterns = [p for p in itertools.product(l, repeat=3)]
    
    
    max_trends = []
    for i in range(len(df['Year'])):
        l = list(dfIndustryTrends.iloc[i])
        m = max(set(l), key = l.count)
        no_cnt = l.count('0')
        up_cnt = l.count('1')
        down_cnt = l.count('-1')
        # Tie
        if(m == '0' and (no_cnt == up_cnt or no_cnt == down_cnt)):
            max_trends.append('0')
                
        elif (m == '1' and (up_cnt == no_cnt or up_cnt == down_cnt)):
            max_trends.append('0')
        elif (m == '-1' and (down_cnt == no_cnt or down_cnt == up_cnt)):
            max_trends.append('0')
        else:
            max_trends.append(m)
    
    max_trends = max_trends[:133]
    #print(max_trends)
    if (period == 'yearly'):
        filtered_trend = max_trends[:7]
    elif (period == 'half-yearly'):
        filtered_trend = max_trends[7:21]
    elif (period == 'quarterly'):
        filtered_trend = max_trends[21:49]
    elif (period == 'monthly'):
        filtered_trend = max_trends[49:133]
        
    
    
    for pattern in patterns:
        trend_map = {'0' : str(pattern[0]), '1' : str(pattern[1]), '-1' : str(pattern[-1])}
        expected_trend = [trend_map[i] for i in crude_oil_trend]
        span = len(filtered_trend)
        trend_score = 0
        for j in range(span):
            if filtered_trend[j] == expected_trend[j]:
                trend_score += 1
        trend_score = trend_score/span
        
        if(trend_score > threshold):
            print(pattern, trend_score)
        
        
        
    
for industry in industry_company_map.keys():
    print("Yearly")
    correlation_trend_analyse_max(industry, period = "yearly")
    print("Half Yearly")
    correlation_trend_analyse_max(industry, period = "half-yearly")
    print("Quarterly")
    correlation_trend_analyse_max(industry, period = "quarterly")
    print("Monthly")
    correlation_trend_analyse_max(industry, period = "monthly")
    
    


Yearly
Refineries/ Petro-Products 7
Half Yearly
Refineries/ Petro-Products 7
(0, 0, -1) 0.7142857142857143
Quarterly
Refineries/ Petro-Products 7
Monthly
Refineries/ Petro-Products 7
Yearly
Fibres & Plastics 7
Half Yearly
Fibres & Plastics 7
(0, 0, 0) 0.7142857142857143
(0, 0, 1) 0.7142857142857143
(0, 0, -1) 0.8571428571428571
(0, 1, -1) 0.7857142857142857
(0, -1, -1) 0.7142857142857143
Quarterly
Fibres & Plastics 7
Monthly
Fibres & Plastics 7
Yearly
Agrochemicals 19
(0, 0, 1) 0.7142857142857143
(0, 1, 1) 0.7142857142857143
Half Yearly
Agrochemicals 19
Quarterly
Agrochemicals 19
Monthly
Agrochemicals 19
(0, 0, 0) 0.7261904761904762
Yearly
Oil Marketing & Distribution 8
(0, 0, 1) 0.7142857142857143
(0, 1, 1) 0.7142857142857143
Half Yearly
Oil Marketing & Distribution 8
Quarterly
Oil Marketing & Distribution 8
Monthly
Oil Marketing & Distribution 8
Yearly
Plastic Products 38
(0, 0, 1) 0.7142857142857143
(0, -1, 1) 0.7142857142857143
Half Yearly
Plastic Products 38
(0, 0, -1) 0.714285714

In [26]:
### Have to add the 2nd hypothesis

import itertools
def correlation_trend_analyse_oneTrendAssumption_max(industry, period, threshold = 0.7):
    print(industry, len(industry_company_map[industry]))
    df = dfStrong[['Year', 'CRUDEOIL']+industry_company_map[industry]]
    crude_oil_trend = list(df['CRUDEOIL'])[:133]
    dfIndustryTrends = df[industry_company_map[industry]]
    
    l = [0, 1, -1]
    patterns = [p for p in itertools.product(l, repeat=3)]
    
    max_trends = []
    for i in range(len(df['Year'])):
        l = list(dfIndustryTrends.iloc[i])
        m = max(set(l), key = l.count)
        no_cnt = l.count('0')
        up_cnt = l.count('1')
        down_cnt = l.count('-1')
        # Tie
        if(m == '0' and (no_cnt == up_cnt or no_cnt == down_cnt)):
            max_trends.append('0')
                
        elif (m == '1' and (up_cnt == no_cnt or up_cnt == down_cnt)):
            max_trends.append('0')
        elif (m == '-1' and (down_cnt == no_cnt or down_cnt == up_cnt)):
            max_trends.append('0')
        else:
            max_trends.append(m)
    
    max_trends = max_trends[:133]
    #print(max_trends)
    if (period == 'yearly'):
        filtered_trend = max_trends[:7]
    elif (period == 'half-yearly'):
        filtered_trend = max_trends[7:21]
    elif (period == 'quarterly'):
        filtered_trend = max_trends[21:49]
    elif (period == 'monthly'):
        filtered_trend = max_trends[49:133]
        
    l = ['0', '1', '-1']
    trend_patterns = {'0' : l, '1' : l, '-1' : l}
    
    for key in trend_patterns.keys():
        for i in l:
            trend_score = 0
            span = len(filtered_trend)
            for k in range(span):
                if (filtered_trend[k] == i and crude_oil_trend[k] == key):
                    trend_score += 1
                elif (filtered_trend[k] != i and crude_oil_trend[k] == key):
                    trend_score += 0
                else:
                    trend_score += 0
            trend_score = trend_score/span
            
            if (trend_score >= threshold):
                print(key,i,trend_score)
                
for industry in industry_company_map.keys():
    print("Yearly")
    correlation_trend_analyse_oneTrendAssumption_max(industry, period = "yearly")
    print("Half Yearly")
    correlation_trend_analyse_oneTrendAssumption_max(industry, period = "half-yearly")
    print("Quarterly")
    correlation_trend_analyse_oneTrendAssumption_max(industry, period = "quarterly")
    print("Monthly")
    correlation_trend_analyse_oneTrendAssumption_max(industry, period = "monthly")
    
        




Yearly
Refineries/ Petro-Products 7
Half Yearly
Refineries/ Petro-Products 7
Quarterly
Refineries/ Petro-Products 7
Monthly
Refineries/ Petro-Products 7
Yearly
Fibres & Plastics 7
Half Yearly
Fibres & Plastics 7
Quarterly
Fibres & Plastics 7
Monthly
Fibres & Plastics 7
Yearly
Agrochemicals 19
Half Yearly
Agrochemicals 19
Quarterly
Agrochemicals 19
Monthly
Agrochemicals 19
Yearly
Oil Marketing & Distribution 8
Half Yearly
Oil Marketing & Distribution 8
Quarterly
Oil Marketing & Distribution 8
Monthly
Oil Marketing & Distribution 8
Yearly
Plastic Products 38
Half Yearly
Plastic Products 38
Quarterly
Plastic Products 38
Monthly
Plastic Products 38
Yearly
Oil Equipment & Services 4
Half Yearly
Oil Equipment & Services 4
Quarterly
Oil Equipment & Services 4
Monthly
Oil Equipment & Services 4
Yearly
Petrochemicals 5
Half Yearly
Petrochemicals 5
Quarterly
Petrochemicals 5
Monthly
Petrochemicals 5
Yearly
Integrated Oil & Gas 2
Half Yearly
Integrated Oil & Gas 2
Quarterly
Integrated Oil & Gas 2